In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN' , 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
cutoff_value = 500
application_types_to_replace = []

# Get the value counts of each application type
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Create a list of application types to be replaced
for app_type, count in application_type_counts.items():
    if count < cutoff_value:
        application_types_to_replace.append(app_type)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

# You may find it helpful to look at CLASSIFICATION value counts >1
filtered_classification_counts = classification_counts[classification_counts > 1]

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000
classifications_to_replace = []

# Get the value counts of each classification
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Create a list of classifications to be replaced
for cls, count in classification_counts.items():
    if count < cutoff_value:
        classifications_to_replace.append(cls)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
#Train model with random forest to check the feature importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder


# Encode categorical columns
categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
                      'USE_CASE', 'ORGANIZATION', 'INCOME_AMT',
                      'SPECIAL_CONSIDERATIONS']

for column in categorical_columns:
    label_encoder = LabelEncoder()
    application_df[column] = label_encoder.fit_transform(application_df[column])

# Separate features and target
X = application_df.drop('IS_SUCCESSFUL', axis=1)
y = application_df['IS_SUCCESSFUL']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# Train a random forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

# Get feature importances
importances = rf_clf.feature_importances_

# Create a DataFrame to display feature importances
feature_importances_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# Display feature importances
print(feature_importances_df)


                  Feature  Importance
8                 ASK_AMT    0.396623
1             AFFILIATION    0.266157
0        APPLICATION_TYPE    0.135102
2          CLASSIFICATION    0.081383
4            ORGANIZATION    0.050343
6              INCOME_AMT    0.042086
3                USE_CASE    0.027089
7  SPECIAL_CONSIDERATIONS    0.000968
5                  STATUS    0.000248


In [7]:
# Select the most important features
important_features = feature_importances_df.head(5)['Feature'].values
X_important = application_df[important_features]

# Get dummy variables for categorical features
X_important_dummies = pd.get_dummies(X_important, drop_first=True)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_important_dummies, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert y_train and y_test to numpy arrays
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()



In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Determine the number of input features
input_features = X_train.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=21, activation='relu', input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=7, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 21)                126       
_________________________________________________________________
dense_13 (Dense)             (None, 7)                 154       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 8         
Total params: 288
Trainable params: 288
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100)

Train on 27439 samples
Epoch 1/100
27439/27439 [==============================] - 1s 53us/sample - loss: 0.6151 - accuracy: 0.6858
Epoch 2/100
27439/27439 [==============================] - 1s 32us/sample - loss: 0.5891 - accuracy: 0.7169
Epoch 3/100
27439/27439 [==============================] - 1s 34us/sample - loss: 0.5840 - accuracy: 0.7206
Epoch 4/100
27439/27439 [==============================] - 1s 35us/sample - loss: 0.5809 - accuracy: 0.7220
Epoch 5/100
27439/27439 [==============================] - 1s 32us/sample - loss: 0.5777 - accuracy: 0.7219
Epoch 6/100
27439/27439 [==============================] - 1s 33us/sample - loss: 0.5744 - accuracy: 0.7220
Epoch 7/100
27439/27439 [==============================] - 1s 33us/sample - loss: 0.5717 - accuracy: 0.7229
Epoch 8/100
27439/27439 [==============================] - 1s 32us/sample - loss: 0.5693 - accuracy: 0.7237
Epoch 9/100
27439/27439 [==============================] - 1s 32us/sample - loss: 0.5678 - accuracy: 0.7241
Epoch

27439/27439 [==============================] - 2s 71us/sample - loss: 0.5566 - accuracy: 0.7305
Epoch 77/100
27439/27439 [==============================] - 2s 69us/sample - loss: 0.5559 - accuracy: 0.7297
Epoch 78/100
27439/27439 [==============================] - 2s 57us/sample - loss: 0.5556 - accuracy: 0.7300
Epoch 79/100
27439/27439 [==============================] - 1s 53us/sample - loss: 0.5555 - accuracy: 0.7289
Epoch 80/100
27439/27439 [==============================] - 2s 74us/sample - loss: 0.5555 - accuracy: 0.7302
Epoch 81/100
27439/27439 [==============================] - 1s 54us/sample - loss: 0.5557 - accuracy: 0.7299
Epoch 82/100
27439/27439 [==============================] - 2s 55us/sample - loss: 0.5555 - accuracy: 0.7309
Epoch 83/100
27439/27439 [==============================] - 2s 62us/sample - loss: 0.5555 - accuracy: 0.7292
Epoch 84/100
27439/27439 [==============================] - 3s 96us/sample - loss: 0.5552 - accuracy: 0.7295
Epoch 85/100
27439/27439 [======

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test ,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6860/1 - 0s - loss: 0.4539 - accuracy: 0.7242
Loss: 0.5655748230375285, Accuracy: 0.7241982221603394


In [18]:
#try with more neurons and more layers

input_features = X_train.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))


# Third hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Fourth Layer
nn.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 128)               768       
_________________________________________________________________
dense_16 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_17 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_18 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 17        
Total params: 11,649
Trainable params: 11,649
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train, y_train, epochs=100)


Train on 27439 samples
Epoch 1/100
27439/27439 [==============================] - 3s 102us/sample - loss: 0.5645 - accuracy: 0.7263
Epoch 2/100
27439/27439 [==============================] - 2s 84us/sample - loss: 0.5625 - accuracy: 0.7259
Epoch 3/100
27439/27439 [==============================] - 2s 87us/sample - loss: 0.5613 - accuracy: 0.7272
Epoch 4/100
27439/27439 [==============================] - 2s 85us/sample - loss: 0.5614 - accuracy: 0.7273
Epoch 5/100
27439/27439 [==============================] - 2s 86us/sample - loss: 0.5602 - accuracy: 0.7272
Epoch 6/100
27439/27439 [==============================] - 2s 88us/sample - loss: 0.5590 - accuracy: 0.7279
Epoch 7/100
27439/27439 [==============================] - 2s 85us/sample - loss: 0.5588 - accuracy: 0.7287
Epoch 8/100
27439/27439 [==============================] - 2s 85us/sample - loss: 0.5587 - accuracy: 0.7303
Epoch 9/100
27439/27439 [==============================] - 2s 85us/sample - loss: 0.5579 - accuracy: 0.7302
Epoc

27439/27439 [==============================] - 3s 104us/sample - loss: 0.5509 - accuracy: 0.7336
Epoch 76/100
27439/27439 [==============================] - 3s 104us/sample - loss: 0.5508 - accuracy: 0.7336
Epoch 77/100
27439/27439 [==============================] - 3s 102us/sample - loss: 0.5510 - accuracy: 0.7336
Epoch 78/100
27439/27439 [==============================] - 3s 100us/sample - loss: 0.5510 - accuracy: 0.7333
Epoch 79/100
27439/27439 [==============================] - 3s 104us/sample - loss: 0.5510 - accuracy: 0.7341
Epoch 80/100
27439/27439 [==============================] - 3s 104us/sample - loss: 0.5509 - accuracy: 0.7341
Epoch 81/100
27439/27439 [==============================] - 3s 100us/sample - loss: 0.5510 - accuracy: 0.7336
Epoch 82/100
27439/27439 [==============================] - 3s 105us/sample - loss: 0.5511 - accuracy: 0.7330
Epoch 83/100
27439/27439 [==============================] - 3s 101us/sample - loss: 0.5512 - accuracy: 0.7329
Epoch 84/100
27439/2743

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test ,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

6860/1 - 0s - loss: 0.4703 - accuracy: 0.7232
Loss: 0.5682231106320206, Accuracy: 0.7231778502464294
